In [1]:
from diwang_feature_exrtaction import *

C:\Users\Administrator\Documents\yh.wang\2\diwang_feature_exrtaction.py:194: RuntimeWarning: invalid value encountered in true_divide
  W = Weight / np.tile(reshape(sum(Weight, 2), (Num, TestData.shape[1] - 1, 1)), (1, 1, Nu))


In [ ]:
# 目标是输入序列，之后补0

In [10]:
import pandas as pd
from torch.utils.data import Dataset
import torch
import numpy as np

In [90]:
import torch.nn as nn

In [3]:
DATA_PATH ="./Data_FD003/preprocessed data/"
attribute = ['Unit','T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30',
                  'phi', 'NRf', 'NRc', 'BPR', 'htBleed', 'W31', 'W32']
df_train = pd.read_csv(DATA_PATH + 'TD_data.csv',names=attribute, header=None)

In [80]:
class AircraftDataset_expend(AircraftDataset):  # 截断原有的数据集，获得海量的数据
    def __init__(self, df, add_zero):
        super().__init__(df)
        self.add_zero = add_zero
        self.cut_data()

    def cut_data(self):
        lenth=super().__len__()
        input_signal=[]
        RUL=[]
        for unit in range(lenth):
            unit_input=super().__getitem__(unit)["input"]
            unit_life=super().__getitem__(unit)["lifetime"]
            if self.add_zero:
                for time in range(3,unit_life):
                    input_tensor = torch.zeros(525, 14, dtype=torch.float)
                    input_tensor[0:time] = unit_input[0:time]
                    unit_RUL=unit_life-time
                    input_signal.append(input_tensor)
                    RUL.append(unit_RUL)
            else:
                for time in range(3,unit_life):
                    input_tensor = unit_input[0:time]
                    unit_RUL=unit_life-time
                    input_signal.append(input_tensor)
                    RUL.append(unit_RUL)

        self.RUL=np.array(RUL)
        self.input_signal=input_signal

    def __len__(self):
        return len(self.RUL)

    def __getitem__(self, idx):
        data = {
            "input": self.input_signal[idx],
            "RUL": torch.tensor(self.RUL[idx],dtype=torch.int64)
        }

        return data

In [86]:
train_data_expend=AircraftDataset_expend(df_train,False)

In [89]:
train_data_expend[0]

{'input': tensor([[ 9.3613e-01,  3.0634e-01,  2.9954e-01, -7.9379e-01, -5.8098e-01,
           1.4285e-01,  5.1867e-01, -2.2405e-01,  1.6889e-01,  2.1825e-01,
           1.1385e+00,  4.3863e-01,  3.6442e-01, -8.4621e-01],
         [ 1.0359e+00,  4.5287e-01,  3.0333e-01, -4.4278e-01, -2.0905e-01,
           1.2649e-01,  3.6916e-01, -9.8438e-02,  2.7012e-01,  5.2459e-01,
           1.2638e+00,  7.5099e-01, -3.0534e-01, -3.6485e-01],
         [ 7.9116e-01,  2.0638e-01,  8.0941e-01, -5.2620e-01,  3.5150e-04,
           4.3203e-01,  3.4586e-01, -4.8677e-01,  1.1415e-01,  4.2597e-01,
           7.9141e-01,  4.3863e-01, -7.5612e-01, -7.9190e-01]]),
 'RUL': tensor(256)}

# 测试pad_GRU

In [104]:
class AircraftDataset_expend_norul(AircraftDataset_expend):
    def __init__(self, df):
        super().__init__(df,add_zero=False)
    def __getitem__(self, idx):

        return self.input_signal[idx]

In [91]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence,pack_sequence,pad_packed_sequence

In [105]:
def collate_fn(data):
    data.sort(key=lambda x: len(x), reverse=True)
    data= pack_sequence(data)
    return data

In [106]:
train_data_expend_norul=AircraftDataset_expend_norul(df_train)

In [107]:
train_data_expend_loader = DataLoader(train_data_expend_norul, 
                                      batch_size=512, 
                                      shuffle=True,
                                      drop_last=True,
                                      collate_fn=collate_fn)

In [130]:
class CFG:
    seed=42
    input_dim=3
    input_feature=18
    num_workers=2
    hidden_size=64
    batch_size=2048
    num_classes=1
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    optimizer="Adam"
    scheduler="ExponentialLR"
    lr= 1e-1
    epoches=10000
    kernel=True
    print_training_process=True
    sc_Gamma=0.999#指数型学习率衰减曲线
    decay = 1e-5
    num_in_feature_classes = 3
    warmup_prop=0.1
    jump_out_value=64

    ## autoencoder
    ae_batch_size=512
    ae_input_layer=14
    ae_hidden_layer=16

In [153]:
class GRUAutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_layer=CFG.ae_input_layer
        self.hidden_layer=CFG.ae_hidden_layer
        self.encoder_GRU = nn.GRU(self.input_layer, self.hidden_layer, batch_first=True)
        self.decoder_GRU = nn.GRU(self.hidden_layer, self.input_layer, batch_first=True)
        
    def forward(self, x):
        x,h0 = self.encoder_GRU(x,
                                torch.zeros(1, CFG.ae_batch_size, CFG.ae_hidden_layer).to(CFG.device))
        decoded_output, hidden = self.decoder_GRU(x, 
                                           torch.zeros(1,CFG.ae_batch_size,CFG.ae_input_layer).to(CFG.device))
        return decoded_output,h0


In [154]:
aemodel=GRUAutoEncoder().to(CFG.device)

In [155]:
loss_func = nn.MSELoss()

In [156]:
aemodel(data.to(CFG.device))

(PackedSequence(data=tensor([[-0.0090, -0.0741, -0.0595,  ..., -0.0164,  0.2473,  0.0415],
         [-0.0090, -0.0741, -0.0595,  ..., -0.0164,  0.2473,  0.0415],
         [-0.0653, -0.0968,  0.0393,  ...,  0.0016,  0.0835,  0.0666],
         ...,
         [ 0.1249, -0.3134, -0.6346,  ..., -0.4069,  0.7393, -0.0185],
         [ 0.1441, -0.3109, -0.6428,  ..., -0.4095,  0.7379, -0.0150],
         [ 0.1547, -0.3090, -0.6472,  ..., -0.4071,  0.7427, -0.0157]],
        device='cuda:0', grad_fn=<CudnnRnnBackward0>), batch_sizes=tensor([512, 512, 512, 508, 502, 499, 497, 495, 493, 492, 490, 487, 482, 480,
         477, 476, 474, 471, 468, 468, 464, 462, 459, 458, 456, 455, 453, 452,
         447, 446, 445, 444, 440, 436, 433, 430, 428, 426, 422, 419, 416, 412,
         410, 407, 404, 401, 399, 397, 393, 392, 388, 385, 384, 382, 381, 381,
         378, 376, 374, 374, 373, 372, 370, 369, 367, 365, 363, 361, 359, 357,
         352, 348, 343, 341, 339, 336, 334, 329, 326, 326, 325, 324, 322, 320,

In [128]:
gru(data,torch.zeros(1, 512, 10))[1].shape

torch.Size([1, 512, 10])

In [125]:
torch.FloatTensor(1,2)

tensor([[1.4013e-45, 0.0000e+00]])